<a href="https://colab.research.google.com/github/Maksimka9/clustering/blob/main/preparing_data_by_sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Траектории студентов

Загрузка библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

Загрузка файла

In [2]:
def load_data(filename):
    df = pd.read_excel(filename)
    return df

In [29]:
filename = r'https://github.com/Maksimka9/clustering/blob/main/full_data.xlsx?raw=true'

data = load_data(filename)


KeyboardInterrupt: ignored

Удаляем точки в данных и пробелы. Приводим все значения к одному регистру

In [148]:
new_data = data[data['sem'] == 8].copy()
del new_data['sem']
print(new_data.shape)

(25418, 8)


In [149]:
# Нужно использовать не просто replace, а str.replace!
# Удалить внутренности скобок. Скобки приходится экранировать с помощью \, иначе они считаются частью регулярного выражения.
new_data['subj'] = new_data['subj'].str.replace('\(.*\)', '', regex=True)

In [150]:
new_data['subj'] = new_data['subj'].str.replace('МДК', '', regex=True)

In [151]:
# Удалить цифры
new_data['subj'] = new_data['subj'].str.replace('\d+', '', regex=True)

In [152]:
# Удаление точек
new_data['subj'] = new_data['subj'].str.replace('\.', '', regex=True)

In [153]:
# Удаление пробелов в начале и конце каждого значения
new_data['subj'] = new_data['subj'].str.strip()

In [154]:
# Преведение к нижнем регистру
# Тут тоже через str.lower!
# Добавил присвоение
new_data['subj'] = new_data['subj'].str.lower()

In [155]:
del new_data['filial']

In [156]:
del new_data['sttext']

In [157]:
del new_data['formobtext']

In [158]:
del new_data['grouptext']

In [159]:
del new_data['yearin']

Преобразование предметов в колонки для анализа оценок по предметам. Если предмета нет, то -1

In [160]:
new_data['mark'] = new_data['mark'].astype('str').str.replace('0', '1', regex=True).astype('int')

In [161]:
# Замена оценок, которые больше 5, на 1
new_data[new_data['mark'] > 5] = 1

In [162]:
# One-hot-encoding столбца subj
new_data = new_data.join(pd.get_dummies(new_data.subj, prefix='subj'))

In [163]:
# Размер таблицы после one-hot-encoding
new_data.shape

(25418, 292)

In [164]:
# Умножение Названий предметов на оценку

In [165]:
cols = new_data.columns.str.startswith('subj_')
print(cols)
print(len(cols))

[False False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [166]:
from tqdm import tqdm

for c in tqdm(new_data.columns[cols]):
    new_data[c] = new_data[c].astype('int').multiply(new_data['mark'], axis=0)

100%|██████████| 289/289 [00:00<00:00, 351.20it/s]


In [167]:
del new_data['mark']

In [168]:
new_data.shape

(25418, 291)

In [169]:
# Для группировки данных по студенту/семестру должно сработать это:
new_data1 = new_data.groupby('hash').sum()

In [170]:
new_data1

,subj_автоматизация производственных процессов в машиностроении,subj_автоматизированные информационные системы,subj_автоматизированые информационные системы,subj_администрирование логистических систем,subj_анализ данных в области рекламы и связей с общественностью,subj_анализ финансовой отчетности,subj_антикризисное управление,subj_аттестация в области защиты информации,subj_аудит,subj_аудит качества,subj_банковские и платежные информационные системы,subj_безопасность банковских и платежных ис,subj_безопасность жизнедеятельности,subj_безопасность информационных систем,subj_бенчмаркинг как инструмент управления качеством и инновациями,subj_бенчмаркинг как этап управления качеством и инновациями,subj_бизнес-консультирование,subj_бизнес-планирование,subj_бухгалтерский финансовый учет,subj_бюджетная система рф,subj_внешнеэкономическая деятельность,subj_выполнение проекта в материале,"subj_выполнение работ по одной или нескольким профессиям рабочих, должностям служащих",subj_выполнение художественно-конструкторских проектов в материале,subj_вышивка и роспись по ткани,subj_гидрогазоаэродинамика,subj_государственная аттестация,subj_государственная и муниципальная служба,subj_государственные и муниципальные финансы,subj_государственный экзамен,subj_декларирование товаров,subj_декларирование товаров и транспортных средств,"subj_деньги, кредит, банки",subj_диагностика и оценка персонала,subj_дизайн среды,subj_защита выпускной квалификационной работы,subj_защищённые информационные системы банковской деятельности,subj_измерение и анализ эффективности и качества,subj_иллюстрация,subj_инвестиционный менеджмент,...,subj_управление инновационной деятельностью,subj_управление инновационными проектами,subj_управление качеством,subj_управление качеством в ракетно-космической отрасли,subj_управление организационными изменениями,subj_управление персоналом,subj_управление персоналом в таможенных органах,subj_управление региональной экономикой,subj_управление рисками в логистических системах,subj_управление таможенными органами,subj_управление финансами в логистических системах,subj_управляющие программы для обработки заготовок на металлорежущем и аддитивном оборудовании,subj_участие во внедрении технологических процессов изготовления деталей машин и осуществление технического контроля,subj_учебная практика,subj_учебная: практика по получению первичных профессиональных умений и навыков,subj_учебная: технологическая практика,subj_учет и анализ внешней экономической деятельности,subj_физическая культура,subj_финансы бюджетных организаций,subj_фирменный стиль,subj_формирование имиджа в органах власти,subj_экологические основы природопользования,subj_экология,subj_экономика и организация производства,subj_экономика и организация производства в ракетно-космической отрасли,subj_экономика информационной безопасности,subj_экономика машиностроительного производства,subj_экономика организации,subj_экономика отраслевых рынков,subj_экономика предпринимательства,subj_экономическая безопасность,subj_экономическая оценка эффективности деятельности,subj_экономическая устойчивость предприятия,subj_экономический анализ,subj_экспертные системы в управлении,subj_эксплуатационные материалы ракетно-космической техники,subj_элективные курсы по физической культуре и спорту,subj_электротехника и электроника,subj_этика государственной и муниципальной службы,subj_этика деловых отношений
hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0x000DE9673F941872AFFC6EF39B3B0EEAC868BA3B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0x00153956450F17DDD7C0610CF52E1476DE4E65E8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0x0019D4D822B0D9A3DD3336438904749E38147903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [171]:
scv_data = new_data1.to_csv('E:\Максим\Дипломный проект\prepared_data_sem_8.csv', index=False)